# Lei de Little 

Número médio de clientes a longo prazo é $L$

$ L = \lambda \cdot W $

onde $\lambda$ é a taxa média efetiva de chegada a longo prazo e $W$ é o tempo médio que um cliente passa no sistema. 

In [1]:
include("utils.jl")
include("cc_os_delay.jl")

cc_os_delay (generic function with 1 method)

In [ ]:
# Objective function Cost
c_z = 1.5      # weight of served clients
c_ser = 2      # weight of server cost  
c_L = 1

# Objective function
lin_fobj(S, dr, Cin, phi, Z, L) = c_ser*sum(S) - c_z*sum(Z) +c_L*sum(L)

# Parameters
N = 10            # numner of examples
horiz = 50        # total horizon

# bounds
XM = 5            # max queue length 
YM = 10           # max buffer legnth before dropping calls
phiM = 9          # max adimission to queue

struct bounds
    XM
    YM
    phiM
    serM  # different for each model
    tserM # only defined for the model with delay
end

# iniital conditions
X0 = 4
Y0 = 6
L0 = 0
Z0 = 1

struct initial_conditions
    X0
    Y0
    L0
    Z0
end
ic = initial_conditions(X0, Y0, L0, Z0)

createDemands = true
if createDemands
    d_mat = demand_generator_mat(1, horiz, 14, "uniform", 1)
    a_mat = demand_generator_mat(1, horiz, 1,"uniform",0.5)  #voltar para 2 depois
else
    d_fn = "..//CC_simple//d_mat_Thu_22_May_2025_19_39_50.txt";
    a_fn = "..//CC_simple//a_mat_Thu_22_May_2025_19_39_50.txt";
    d_mat = DelimitedFiles.readdlm(d_fn);
    a_mat = DelimitedFiles.readdlm(a_fn);
    
end

d = d_mat[1:horiz, 1];  # demand for incoming calls
a = a_mat[1:horiz, 1];  # abandonment for calls

struct result
    id
    X
    Y
    Z
    L
    n
    Q
    dr
    phi
    Cin
    Cout
    S
    Sl
    Sa
    Sst
    Sc
    Sin
    Saux
    b_opt
    J
    c_ser
    c_blr
end

nl = true # true for non linear obj func and false for linear obj func

# costs for the non linear obj func
c_blro = 140
c_sero = 1

serMo = 5  # number of servers

bdso = bounds(XM, YM, phiM, serMo, 0);


tserM = 3         # max delay in service time
tm = (((tserM+1)*(tserM+2)/2)-1)/tserM # average service time

c_serd = c_sero/tm
c_blrd = c_blro

serMd = serMo*tm         # number of servers
serMd = Int(serMd)
bdsd = bounds(XM, YM, phiM, serMd, tserM)

rnd_tser = true
df_input = df_input_generator(horiz, serMd, tserM, rnd_tser);
   

In [3]:
optimal, X, Y, Z, L, n, Q, dr, phi, Cin, Cout, S, Sl, Sa, Sst, Sc, Sin, Saux, b_opt, J = cc_os_delay(ic, bdsd, a, d, df_input, lin_fobj);
result_os_del = result("os_del", X, Y, Z, L, n, Q, dr, phi, Cin, Cout, S, Sl, Sa, Sst, Sc, Sin, Saux, b_opt, J, c_serd, c_blrd);

In [5]:
lambda = (sum(d)-sum(dr))/horiz                     # average effective arrival rate
L = (sum(X[1:horiz])+sum(Y[1:horiz])+sum(S))/horiz  # average number of clients in the system
W = L/lambda                                        # average waiting time

println("lambda: ", lambda)
println("L: ", L)
println("W: ", W)

lambda: 4.82
L: 27.24
W: 5.651452282157676
